## 1. Data Loading and Preparation

In [1]:
import pandas as pd

**Data Dictionary**

This section clarifies the meaning of our columns, especially the coded features.

* **age_years**: Age of the patient (in years).
* **gender**: 1: Male, 2: Female
* **height_cm**: Height in centimeters.
* **weight_kg**: Weight in kilograms.
* **systolic_bp**: Systolic blood pressure (the "upper" number).
* **diastolic_bp**: Diastolic blood pressure (the "lower" number).
* **cholesterol**: 1: Normal, 2: Above Normal, 3: Well Above Normal
* **glucose**: 1: Normal, 2: Above Normal, 3: Well Above Normal
* **is_smoker**: 0: No, 1: Yes
* **is_alcoholic**: 0: No, 1: Yes
* **is_active**: 0: No, 1: Yes
* **target**: 0: No cardiovascular disease, 1: Presence of cardiovascular disease

In [13]:
df = pd.read_csv('../data/cardio_train.csv', sep=';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [14]:
# rename columns for better clarity
column_mapping = {
    'age': 'age_days',
    'height': 'height_cm',
    'weight': 'weight_kg',
    'ap_hi': 'systolic_bp',
    'ap_lo': 'diastolic_bp',
    'gluc': 'glucose',
    'smoke': 'is_smoker',
    'alco': 'is_alcoholic',
    'active': 'is_active',
    'cardio': 'target'
}

df = df.rename(columns=column_mapping)

In [15]:
# convert age from days to years
df['age_years'] = (df['age_days'] / 365.25).round().astype(int)

df = df.drop(columns=['id', 'age_days'])

In [16]:
final_columns = [
    'age_years',
    'gender',
    'height_cm',
    'weight_kg',
    'systolic_bp',
    'diastolic_bp',
    'cholesterol',
    'glucose',
    'is_smoker',
    'is_alcoholic',
    'is_active',
    'target'
]

df = df[final_columns]

In [17]:
df.head()

,age_years,gender,height_cm,weight_kg,systolic_bp,diastolic_bp,cholesterol,glucose,is_smoker,is_alcoholic,is_active,target
0,50,2,168,62.0,110,80,1,1,0,0,1,0
1,55,1,156,85.0,140,90,3,1,0,0,1,1
2,52,1,165,64.0,130,70,3,1,0,0,0,1
3,48,2,169,82.0,150,100,1,1,0,0,1,1
4,48,1,156,56.0,100,60,1,1,0,0,0,0


## 2. Exploratory Data Analysis (EDA)

In [18]:
print("Target Variable Distribution (0 = No Disease, 1 = Disease):")
target_distribution = df['target'].value_counts(normalize=True)
print(target_distribution)

Target Variable Distribution (0 = No Disease, 1 = Disease):
target
0    0.5003
1    0.4997
Name: proportion, dtype: float64


In [20]:
from sklearn.metrics import mutual_info_score

In [21]:
categorical_features = [
    'gender', 
    'cholesterol', 
    'glucose', 
    'is_smoker', 
    'is_alcoholic', 
    'is_active'
]

numerical_features = [
    'age_years', 
    'height_cm', 
    'weight_kg', 
    'systolic_bp', 
    'diastolic_bp'
]

In [ ]:
# --- 1. Mutual Information (for Categorical) ---
# This tells us how much "information" a feature gives us about the target

def mutual_info_target_score(series):
    return mutual_info_score(series, df['target'])

print("--- Mutual Information (Categorical Features) ---")
# We apply this function to all categorical columns
mi_scores = df[categorical_features].apply(mutual_info_target_score)
print(mi_scores.sort_values(ascending=False))

print("\n")

# --- 2. Correlation (for Numerical) ---
# This tells us how linearly related a feature is to the target
# We use .abs() because a strong negative correlation (-0.8) is just
# as important as a strong positive one (0.8).

print("--- Correlation (Numerical Features) ---")
correlation_scores = df[numerical_features].corrwith(df['target']).abs()
print(correlation_scores.sort_values(ascending=False))

--- Mutual Information (Categorical Features) ---
cholesterol     0.025351
glucose         0.004223
is_active       0.000636
is_smoker       0.000120
gender          0.000033
is_alcoholic    0.000027
dtype: float64


--- Correlation (Numerical Features) ---
age_years       0.237802
weight_kg       0.181660
diastolic_bp    0.065719
systolic_bp     0.054475
height_cm       0.010821
dtype: float64


## 3. Validation Framework

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
# Split the Data ---
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [38]:
# Then, split the 80% (full_train) into 75% (train) and 25% (validation)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [39]:
# get the 'target' column (our y) from each dataframe
y_train = df_train['target'].values
y_val = df_val['target'].values
y_test = df_test['target'].values

# delete the 'target' column from the feature dataframes
df_train = df_train.reset_index(drop=True)

del df_val['target']
df_val = df_val.reset_index(drop=True)

del df_test['target']
df_test = df_test.reset_index(drop=True)


# Check the results ---
print(f"Training set size:   {len(df_train)}")
print(f"Validation set size: {len(df_val)}")
print(f"Test set size:       {len(df_test)}")

Training set size:   42000
Validation set size: 14000
Test set size:       14000
